# Visualize random instances of the ReplicaGrasp dataset.

In [ ]:
# Imports.
import sys, os
sys.path.append('../../')

from flex.tools.utils import get_ground, rotmat2aa
from psbody.mesh.colors import name_to_rgb
from flex.tools.config import Config
from omegaconf import OmegaConf
from psbody.mesh import Mesh
import meshplot as mp
import numpy as np
import random
import smplx
import torch
import mano

random.seed(123)

In [ ]:
# Configurations (objects, receptacles, orientations, instances)
grab_objs = ['binoculars', 'wineglass', 'toothpaste',
             'fryingpan', 'elephant', 'apple', 'toothbrush',
             'cylindersmall', 'train', 'spheresmall', 'stapler',
             'waterbottle', 'pyramidlarge', 'hammer', 'phone', 'airplane',
             'toruslarge', 'piggybank', 'alarmclock', 'pyramidmedium', 'mouse',
             'headphones', 'cylinderlarge', 'spheremedium', 'stamp',
             'cubesmall', 'stanfordbunny', 'torussmall', 'cubelarge', 'watch',
             'spherelarge', 'duck', 'torusmedium', 'flute', 'flashlight',
             'cubemedium', 'scissors', 'gamecontroller', 'teapot', 'banana',
             'cylindermedium', 'knife', 'eyeglasses', 'pyramidsmall', 'lightbulb',
             'bowl', 'camera', 'cup', 'mug', 'hand']
receptacles = ['receptacle_aabb_Chr1_Top1_frl_apartment_chair_01', 'receptacle_aabb_Sofa_frl_apartment_sofa', 'receptacle_aabb_Tbl1_Top1_frl_apartment_table_01', 'receptacle_aabb_Tbl2_Top1_frl_apartment_table_02', 'receptacle_aabb_Tbl2_Top2_frl_apartment_table_02', 'receptacle_aabb_Tbl2_Top3_frl_apartment_table_02', 'receptacle_aabb_TvStnd1_Top1_frl_apartment_tvstand', 'receptacle_aabb_TvStnd1_Top2_frl_apartment_tvstand', 'receptacle_aabb_TvStnd1_Top3_frl_apartment_tvstand', 'receptacle_aabb_WC1_Top1frl_apartment_wall_cabinet_01', 'receptacle_aabb_WC1_Top2frl_apartment_wall_cabinet_01', 'receptacle_aabb_WC1_Top3frl_apartment_wall_cabinet_01', 'receptacle_aabb_WC1_Top4frl_apartment_wall_cabinet_01', 'receptacle_aabb_WC1_Top5frl_apartment_wall_cabinet_01', 'receptacle_aabb_WC1_Top6frl_apartment_wall_cabinet_01', 'receptacle_aabb_WC2_Top1_frl_apartment_wall_cabinet_02', 'receptacle_aabb_WC2_Top2_frl_apartment_wall_cabinet_02', 'receptacle_aabb_WC2_Top3_frl_apartment_wall_cabinet_02', 'receptacle_aabb_WC2_Top4_frl_apartment_wall_cabinet_02', 'receptacle_aabb_WC2_Top5_frl_apartment_wall_cabinet_02', 'receptacle_aabb_WC2_Top6_frl_apartment_wall_cabinet_02', 'receptacle_aabb_bottomfrl_apartment_refrigerator', 'receptacle_aabb_middle_bottomfrl_apartment_refrigerator', 'receptacle_aabb_middle_topfrl_apartment_refrigerator', 'receptacle_aabb_middlefrl_apartment_refrigerator', 'receptacle_aabb_topfrl_apartment_refrigerator', 'receptacle_aabb_shelf_center_bottom_left_kitchenCupboard_01', 'receptacle_aabb_shelf_center_bottom_right_kitchenCupboard_01', 'receptacle_aabb_shelf_center_middle_left_kitchenCupboard_01', 'receptacle_aabb_shelf_center_middle_right_kitchenCupboard_01', 'receptacle_aabb_shelf_center_top_left_kitchenCupboard_01', 'receptacle_aabb_shelf_center_top_right_kitchenCupboard_01', 'receptacle_aabb_shelf_left_bottom_kitchenCupboard_01', 'receptacle_aabb_shelf_left_middle_kitchenCupboard_01', 'receptacle_aabb_shelf_left_top_kitchenCupboard_01', 'receptacle_aabb_shelf_right_bottom_kitchenCupboard_01', 'receptacle_aabb_shelf_right_middle_kitchenCupboard_01', 'receptacle_aabb_shelf_right_top_kitchenCupboard_01', 'receptacle_aabb_counter_left_kitchen_counter', 'receptacle_aabb_counter_right_kitchen_counter', 'receptacle_aabb_drawer_left_bottom_frl_apartment_kitchen_counter', 'receptacle_aabb_drawer_left_top_frl_apartment_kitchen_counter', 'receptacle_aabb_drawer_middle_bottom_frl_apartment_kitchen_counter', 'receptacle_aabb_drawer_middle_middle_frl_apartment_kitchen_counter', 'receptacle_aabb_drawer_middle_top_frl_apartment_kitchen_counter', 'receptacle_aabb_drawer_right_bottom_frl_apartment_kitchen_counter', 'receptacle_aabb_drawer_right_top_frl_apartment_kitchen_counter', 'receptacle_aabb_sink_kitchen_counter']
orients = ['up', 'all']
idxs = list(range(2))

# Load dataset.
recept_dict = dict(np.load('../../data/replicagrasp/receptacles.npz', allow_pickle=1))
dset_info_dict = dict(np.load('../../data/replicagrasp/dset_info.npz', allow_pickle=1))

# Load mesh path.
mesh_pth = '../../data/obj/contact_meshes/'

In [ ]:
i = 0
while i < 10:
    
    # ---> Load data.
    obj_name, ornt, recept, ind = random.choice(grab_objs), random.choice(orients), random.choice(receptacles), random.choice(idxs)
    print(obj_name, ornt, recept, ind)    
    transl_grab, orient_grab, recept_idx = dset_info_dict[f'{obj_name}_{recept}_{ornt}_{ind}']
    recept_v, recept_f = recept_dict[recept][recept_idx][0], recept_dict[recept][recept_idx][1]

    # ---> Visualize receptacle meshes in scene.
    mp_viewer = mp.plot(recept_v, recept_f, name_to_rgb['yellow'])

    # ---> Visualize ground for context (center it to mean of rigid/articulated object).
    grnd_mesh = get_ground(grnd_size=10, offset=0.0)
    xmean, ymean, _ = recept_v.mean(0)
    grnd_mesh.v[:, 0] += xmean
    grnd_mesh.v[:, 1] += ymean
    mp_viewer.add_mesh(grnd_mesh.v, grnd_mesh.f, name_to_rgb['brown'])

    # ---> Visualize object to be grasped in scene.
    object_mesh = Mesh(filename=os.path.join(mesh_pth, obj_name + '.ply'), vscale=1.)
    obj_verts = np.matmul(object_mesh.v, orient_grab.T) + transl_grab
    mp_viewer.add_mesh(obj_verts, object_mesh.f, name_to_rgb['blue'])
    
    i += 1

print("Done!")